In [ ]:
import os
import glob
from PIL import Image
import numpy as np
import PyPDF2
pdf_path = "original"
pgm_path = "pgm"
target_path = "trimed"
os.makedirs(pgm_path, exist_ok=True)
os.makedirs(target_path, exist_ok=True)

pages = {}
for pdf in os.listdir(pdf_path):
    os.makedirs(os.path.join(pgm_path, pdf[:-4]), exist_ok=True)
    os.system("pdftoppm -gray "+os.path.join(pdf_path, pdf)+" "+os.path.join(pgm_path, pdf[:-4], "pgm"))
    lastpix = None
    haslastpix = False

    containpages = []
    currenthold = -1

    filelist = glob.glob(os.path.join(pgm_path, pdf[:-4], '*.pgm'))
    for filename in sorted(filelist, key=lambda s: s.lower()):
        pagenr = filename.rsplit("/",1)[-1].rsplit(".",1)[0].rsplit("-",1)[-1]

        img = Image.open(filename)
        pix = np.array(img)
        img.close()

        if haslastpix:
            isconsecutive = (((lastpix >= pix).sum() / pix.shape[0] / pix.shape[1]) > 0.9995 )
            if not isconsecutive:
                containpages.append(currenthold)

        lastpix = pix
        haslastpix = True
        currenthold = int(pagenr)

    containpages.append(currenthold)
    pages[pdf] = containpages

In [ ]:
for pdf in pages:
    in_file = PyPDF2.PdfFileReader(os.path.join(pdf_path, pdf), 'rb') 
    out_file = PyPDF2.PdfFileWriter()

    for pgNo in range(in_file.numPages):
        if pgNo+1 in pages[pdf]:
            pg = in_file.getPage(pgNo)
            out_file.addPage(pg)

    with open(os.path.join(target_path, pdf), 'wb') as f:
        out_file.write(f)